In [1]:
from bs4 import BeautifulSoup
import json
from selenium import webdriver
import time

In [2]:
# ref: https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
def scroll_down_to_bottom(driver):
    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
#         print("scroll down event")
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Click show more button if any
        show_more_btn = driver.find_element_by_css_selector('#smb')
        try:
#             print(show_more_btn.get_attribute("outerHTML"))
            show_more_btn.click()
        except:
            x = 0

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [3]:
def download_page_internal(driver, url):
    driver.get(url)
    scroll_down_to_bottom(driver)
    elem = driver.find_element_by_xpath("//*")
    page = elem.get_attribute("outerHTML")
    return page

In [4]:
def get_image_urls_from_html(raw_html):
    soup = BeautifulSoup(raw_html, "lxml")
    divs = soup.find_all('div', class_='rg_meta notranslate')
    return [json.loads(div.text)['ou'] for div in divs]

In [5]:
def get_all_items_from_url(url):
    try:
        options = webdriver.ChromeOptions()
        # MacOS
        # options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
        # Linux chrome binary
        options.binary_location = '/usr/bin/google-chrome-stable'
        # comment headless argument for debugging/ visualization
        options.add_argument('headless')
        options.add_argument('window-size=1200x600')

        # add proxy, restarting chrome needed if not work
        http_proxy="localhost:8123"
        options.add_argument('--proxy-server={0}'.format(http_proxy))

        driver = webdriver.Chrome(chrome_options=options)
    
        raw_html = download_page_internal(driver, url)
        return get_image_urls_from_html(raw_html)
    except:
        return []
    finally:
        # close chrome driver by calling quit()
        driver.quit()

In [6]:
links = get_all_items_from_url("https://www.google.com/search?q=Brinjal%20Fruit%20Borer&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&tbs=&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg")

In [7]:
len(links)

789

In [8]:
# # basic operations
# driver.get('https://www.google.com/imghp?hl=en&tab=wi&authuser=0')
# # chrome extension, get xpath of element, http://selectorgadget.com/
# search = driver.find_element_by_css_selector('input#lst-ib')
# click = driver.find_element_by_css_selector('#sfdiv :nth-child(1)')
# search.send_keys("golang")
# click.click()
# # save image
# driver.get_screenshot_as_file('main-page2.png')